In [1]:
import pandas as pd
import math
from tqdm import tqdm_notebook

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.set_option('max_colwidth', 600)

In [8]:
# import play-by-play data
data = pd.read_csv('../data/interim/clean data unverified scores 6.14.19.csv')
data = data.drop('Unnamed: 0', axis=1)

# import matchup data to get scores at the end of each quarter
matchup_df = pd.DataFrame()
weeks = 16
for week in range(1, weeks+1):
    df = pd.DataFrame(pd.read_csv(f'../data/raw/matchups/2018 Week {week}.csv', encoding = "ISO-8859-1"))
    matchup_df = matchup_df.append(df, ignore_index=True, sort=False)
matchup_df['start_date'] = pd.to_datetime(matchup_df['start_date'])

In [3]:
def get_periods(game_id):
    """Return the number of combined quarters and overtime periods in a given game (minimum 4)
    
    Arguments:
    game_id -- an int identifying the game
    """
    return max(len(data[data['id']==game_id].loc[data['play_text'].str.contains('End of')]), 4)

def get_pbp_score(game_id, side, period, pbp=data):
    """Retrieve the score for a period in the pbp dataset identified by the given game_id, side, and period.
    
    Arguments:
    game_id -- an int identifying the game
    side -- a string that is either 'home' or 'away'
    period -- an int identifying the quarter or overtime period needed (indexed at 1)
    pbp -- the name of the play-by-play dataset (default data)
    """
    return int(pbp[pbp['id']==game_id].loc[pbp['period']==period , f'{side}_score'][-1:])

def get_matchup_score(game_id, side, period, matchup=matchup_df):
    """Retrieve the score for a period in the matchup dataset identified by the given game_id, side, and period.
    
    Games in this dataset have a maximum of five periods, so games with more than 1 overtime period
    are not fully verifiable with this verification method.
    
    Arguments:
    game_id -- an int identifying the game
    side -- a string that is either 'home' or 'away'
    period -- an int identifying the quarter or overtime period needed (indexed at 1)
    matchup -- the name of the matchup dataset (default matchup_df)
    """
    if (period > 5):
        return 0
    else:
        score = float(matchup.loc[matchup['id']==game_id, f'{side}_line_scores[{period-1}]'])
        if math.isnan(score):
            return 0
        else:
            return score

def compare_scores(game_id, side):
    """Compare the end-of-period scores of a given game_id and side in the play-by-play and matchup datasets.
    
    Use the helper fuctions get_periods(),  get_pbp_score() and get_matchup_score()
    to retrieve the scores necessary for comparison.
    
    Arguments:
    game_id -- an int identifying the game
    side -- a string that is either 'home' or 'away'
    """
    periods = get_periods(game_id)
    pbp_scores = []
    matchup_scores = []
    for period in range(1, periods+1):
        pbp_score = get_pbp_score(game_id, side, period)
        matchup_score = get_matchup_score(game_id, side, period)
        pbp_scores.append(pbp_score)
        matchup_scores.append(matchup_score)
    difference = dict()
    for i in range(len(pbp_scores)):
        difference[f'q{i+1}'] = pbp_scores[i] - sum(matchup_scores[:i+1])
    difference['id'] = game_id
    difference['side'] = side
    return difference

def return_diffs(df):
    """Return a DataFrame of differences between pbp and matchup data that are not equal to 0.
    
    Arguments:
    df -- a DataFrame of score differences returned from compare_scores()
    """
    diffs = df.loc[(df.q1 != 0.0) | (df.q2 != 0.0) | (df.q3 != 0.0) | (df.q4 != 0.0) | ((df.q5 != 0.0) & (df.q5.notnull()))]
    return diffs

In [4]:
comparison_home = pd.DataFrame([compare_scores(game, 'home') for game in tqdm_notebook(data.id.unique())])

In [5]:
comparison_away = pd.DataFrame([compare_scores(game, 'away') for game in tqdm_notebook(data.id.unique())])

In [7]:
cols = ['id', 'side', *[f'q{i}' for i in range(1,11)]]
comparison_home = comparison_home[cols]
comparison_away = comparison_away[cols]
comparison_all = comparison_home.append(comparison_away)
return_diffs(comparison_all).to_csv('matchup_pbp_score_differences.csv')

In [29]:
return_diffs(comparison_home).to_csv('../data/interim/verification/matchup_score_diffs_home.csv')
return_diffs(comparison_away).to_csv('../data/interim/verification/matchup_score_diffs_away.csv')

In [14]:
matchup_scores = matchup_df[['id', 'week', 'home_team', 'away_team', *[f'home_line_scores[{i}]' for i in range(5)], *[f'away_line_scores[{i}]' for i in range(5)]]].set_index('id', drop=True)
matchup_scores.columns = ['week', 'home_team', 'away_team', *[f'home_score_q{i}' for i in range (1,6)], *[f'away_score_q{i}' for i in range (1,6)]]
matchup_scores.head()


,week,home_team,away_team,home_score_q1,home_score_q2,home_score_q3,home_score_q4,home_score_q5,away_score_q1,away_score_q2,away_score_q3,away_score_q4,away_score_q5
id,,,,,,,,,,,,,
401013357,1,UMass,Duquesne,21,14,21,7,NaN,3,6,0,6,NaN
401014972,1,Rice,Prairie View,13,6,0,12,NaN,0,21,7,0,NaN
401022510,1,Colorado State,Hawai'i,7,0,13,14,NaN,10,13,14,6,NaN
401013437,1,New Mexico State,Wyoming,0,0,0,7,NaN,9,6,7,7,NaN
401020671,1,Ball State,Central Connecticut,7,14,14,7,NaN,0,0,0,6,NaN


In [20]:
side='home'
matchup_scores[[f'{side}_score_q{i}' for i in range (1,6)]].cumsum(axis=1)

,home_score_q1,home_score_q2,home_score_q3,home_score_q4,home_score_q5
id,,,,,
401013357,21.0,35.0,56.0,63.0,NaN
401014972,13.0,19.0,19.0,31.0,NaN
401022510,7.0,7.0,20.0,34.0,NaN
401013437,0.0,0.0,0.0,7.0,NaN
401020671,7.0,21.0,35.0,42.0,NaN
401013328,7.0,35.0,38.0,48.0,NaN
401021651,0.0,14.0,14.0,24.0,NaN
401019470,0.0,10.0,10.0,17.0,NaN
401014979,7.0,28.0,38.0,52.0,NaN
